# Fluent Python
https://github.com/fluentpython/example-code

파이썬 용어집 

https://docs.python.org/ko/3/glossary.html

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np


## CHAPTER 15. Context Managers and else Blocks


### Do This, Then That: else Blocks Beyond if

* for/else
    for 완전히 실행된 후 (break 문으로 멈추지 않으면) else 블록 실행
* while/else
    while 문의 조건이 거짓이 되어 (break 문으로 멈추지 않으면) else 블록 실행
* try/else
    try 예외가 발생하지 않을 때만 else 실행. else 에서 발생한 예외는 처리 되지 않는다. 
    
예외, return, break, continue 문이 복합문의 주요 블럭을 빠져나오게 만들면 else 블록은 실행되지 않는다. 

* EAFP
허락을 구하기보다 용서를 구하는 것이 쉽다. Easier to ask for forgiveness than permission. <br>
가정이 잘못되었을때는 예외를 잡아서 처리하는 스타일 try/chtch

* LBYL
누울자리를 보고 발을 뻗는다. Look before you leap <br>
호출이나 조회하기 전에 조건을 검사한다. if 

파이썬에서는 **EAFP** 사용 


### Context Managers and with Blocks

context manager 의 프로토콜은 \_\_enter\_\_() dhk \_\_exit\_\_() 로 구성됨. <br>
with 문이 시작될 떄 콘텍스트 관리자의 객체의 \_\_enter\_\_() 메서드가 호출. <br>
with 블록의 끝에서 finally 절의 역할을 수행. 


In [2]:
class LookingGlass:
    
    def __enter__(self):
        import sys
        self.original_write = sys.stdout.write # Monkey-patch
        sys.stdout.write = self.reverse_write
        return 'JABBERWOCKY' # as 변수 what 에 문자열 반환 
    
    def reverse_write(self, text):
        self.original_write(text[::-1])
        
    def __exit__(self, exc_type, exc_value, traceback):
        import sys
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print('Please DO NOT divide by zero!')
            return True

In [3]:
with LookingGlass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)
    
what

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


'JABBERWOCKY'

In [4]:
print('Back to normal')

Back to normal


tip) 표준 출력을 가로챌때 sys.stdout 을 다른 객체로 사용한다. 참고 [contextlib.redirect_stdout](https://docs.python.org/3/library/contextlib.html#contextlib.redirect_stdout) 


* \_\_exit\_\_() 의 세가지 인수 
    - exc_type : The exception class (e.g., ZeroDivisionError).
    - exc_value : The exception instance. Sometimes, parameters passed to the exception constructor—such as the error message—can be found in exc_value.args.
    - traceback : A traceback object.3

createive uses of context manager 
- Managing transactions in the sqlite3 module; see “[12.6.7.3. Using the connection as a context manager](https://docs.python.org/3/library/sqlite3.html#using-the-connection-as-a-context-manager)”.
- Holding locks, conditions, and semaphores in threading code; see “[17.1.10. Using locks, conditions, and semaphores in the with statement](https://docs.python.org/3/library/threading.html#using-locks-conditions-and-semaphores-in-the-with-statement)”.
- Setting up environments for arithmetic operations with Decimal objects; see [the decimal.localcontext documentation](https://docs.python.org/3/library/decimal.html#decimal.localcontext).
- Applying temporary patches to objects for testing; see [the unittest.mock.patch function](https://docs.python.org/3/library/unittest.mock.html#patch).

### The contextlib Utilities

contextlib 의 유용한 함수들 

- closing : A function to build context managers out of objects that provide a close() method but don’t implement the \_\_enter\_\_/\_\_exit\_\_ protocol.


- suppress : A context manager to temporarily ignore specified exceptions. 


- @contextmanager: A decorator that lets you build a context manager from a simple generator function, instead of creating a class and implementing the protocol.


- ContextDecorator : A base class for defining class-based context managers that can also be used as function decorators, running the entire function within a managed context.


- ExitStack : A context manager that lets you enter a variable number of context managers. When the with block ends, ExitStack calls the stacked context managers’\_\_exit\_\_ methods in LIFO order (last entered, first exited).

### Using @contextmanager

@contextmanger 로 decorate 된 제너레이터에서 yeild 함수 본체를 두 부분으로 나누기 위해 사용. <br>
yield 문 앞에 있는 모든 코드는 with 블록 앞에서 인터프리터가 \_\_enter\_\_() 를 호출할떄 실행, <br>
yield 문 뒤에 있는 코드는 블록의 마지막에 \_\_exit\_\_() 가 호출될때 실행된다. 



In [25]:
import contextlib

@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write

    def reverse_write(text):
        original_write(text[::-1])
        
    sys.stdout.write = reverse_write
    msg = ''
    try:
        yield 'JABBERWOCKY'
    except ZeroDivisionError:
        msg = 'Please DO NOT divide by zero!'
    finally:
        sys.stdout.write = original_write
    if msg:
        print(msg)
    print("abc")

In [26]:
with looking_glass() as what:
    print('Alice, Kitty and Snowdrop')
    raise BaseException 
    # yield 구문에서 exception 발생했을때 
    # context 객체의 finally 에 close 작업을 넣어 주지 않으면 exit 가 되지 않음 
    print(what)
    
what    

pordwonS dna yttiK ,ecilA


BaseException: 

In [27]:
print("abc")

abc


The \_\_enter\_\_ method of that class:
1. 제너레이터 함수를 호출해서 제너레이터 객체를 보관한다. (객체를 gen 이라고 부름)
2. next(gen) 을 호출해서 yield 키워드 앞까지 실행
3. next(gen) 이 생성한 값을 반환해서 이 값이 as 절의 타깃 변수에 바인딩 되게 함

When the with block terminates, the \_\_exit\_\_ method:
1. exc_type 에 예외가 전달 되었는지 확인. 만일 전달 되었다면, 제너레이터 함수 본체 안에 있는 yield 행에서 gen.throw(exception) 를 실행해서 예외를 발생시킨 것.
2. 예외가 전달 되지 않았다면, next(gen) 을 호출해서 제너레이터 함수 본체 안의 yield 다음 코드를 실행

\_\_exit\_\_() 예외 처리를 완료했음을 True 를 리턴함으로 인터프리터에게 알림. 인터프리터는 예외를 전파하지 않음. 

\_\_exit\_\_() 값을 반환하지 않으면, None 을 받음으로 예외를 전파함 

@contextmanager 데코레이터를 사용하면 동작이 반대가 됨, 예외를 발생시키고 싶은 경우 명시적으로 예외를 발생시켜야함

tip) @contextmanger 를 사용할때에는 어쩔수 없이 yield 문 주변을 try/finally 나 with 문으로 싸야 한다. context manager 의 사용자가 자신의 with 블록 안에서 어떤 일을 할지 모르기 때문이다.